In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torchvision import datasets, models, transforms
import cv2
from PIL import Image
import numpy as np
import pandas as pd
import os
from datetime import datetime

In [2]:
class MyDataset(Dataset):
    """My dataset."""

    def __init__(self, root_dir, class_path, transform=None):
        super(MyDataset, self).__init__()
        self.root_dir = root_dir
        self.class_path = class_path
        self.transform = transform
        
        # get image list
        all_list =  [os.path.splitext(f) for f in os.listdir(self.root_dir)] # jpg + txt
        assert [f for f,e in all_list if e=='.jpg'] == [f for f,e in all_list if e=='.txt'], "num_image != num_txt"
        self.data_list = [f for f,e in all_list if e=='.jpg']
        
        # get class list
        with open(class_path, 'r', encoding='utf-8') as f:
            self.classes = [c for c in f.read().split('\n')]
        self.class_to_idx = {self.classes[i]: i for i in range(len(self.classes))}

    def __len__(self):
        return len(self.data_list)

    def __getitem__(self, idx):
        image_path = os.path.join(self.root_dir, self.data_list[idx]+'.jpg')
        ann_path = os.path.join(self.root_dir, self.data_list[idx]+'.txt')
        img = Image.open(image_path)
        with open(ann_path, 'r') as f:
            anno = [a.split(' ') for a in f.read().strip().split('\n')]
        anno = np.array(anno)
        labels = anno[:,0].astype(np.int)
        bboxes_yolo = anno[:,1:].astype(np.float) # centerX, centerY, w, h [ratio] - yolo style
        bboxes = bboxes_yolo.copy() # x,y,w,h

        if self.transform:
            img = self.transform(img).unsqueeze(0)
            
        bboxes[:,0] = (bboxes_yolo[:,0] - bboxes_yolo[:,2]/2)*img.shape[3] # x
        bboxes[:,1] = (bboxes_yolo[:,1] - bboxes_yolo[:,3]/2)*img.shape[2] # y
        bboxes[:,2] = bboxes_yolo[:,2]*img.shape[3]
        bboxes[:,3] = bboxes_yolo[:,3]*img.shape[2]
        bboxes = bboxes.astype(np.int)

        return img, labels, bboxes

---
# Classification

In [14]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, models, transforms
import cv2
from PIL import Image
import numpy as np
import pandas as pd
import os, shutil
import time
from time import sleep
import subprocess
import matplotlib.pyplot as plt
import seaborn as sns
import torchvision
from sklearn.decomposition import PCA
from sklearn.mixture import GaussianMixture
from torch.distributions.multivariate_normal import MultivariateNormal
from tqdm import tqdm_notebook as tqdm
# import mymodels
# from mymodels.BN_Inception import BN_Inception

def my_cos(x,y):
    _x = F.normalize(x, p=2, dim=1)
    _y = F.normalize(y, p=2, dim=1)
    return _x.matmul(_y.transpose(0,1))

class Flatten(torch.nn.Module):
    """
    torch.nn.Sequential에서 사용가능한 flatten 모듈
    """
    def forward(self, x):
        batch_size = x.shape[0]
        return x.view(batch_size, -1)
    
class OracleModel(nn.Module):
    def __init__(self):
        """Load the pretrained ResNet-152 and replace top fc layer."""
        super(OracleModel, self).__init__()
#         ****torch pretrained net****
#         net = models.shufflenetv2.shufflenet_v2_x1_0(pretrained=True) # 1024
#         net = torch.hub.load('pytorch/vision', 'mobilenet_v2', pretrained=True)
#         net = models.mobilenet_v2(pretrained=True)
#         net = models.densenet201(pretrained=True)
        net = models.resnet50(pretrained=True)

        modules = list(net.children())[:-2]      # resnet conv_5
        self.backbone = torch.nn.Sequential(*modules) # 2048
        avg_pool = torch.nn.AdaptiveAvgPool2d(output_size=(1, 1))
        flatten = Flatten() 
        
#         net = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True, 
#                                                              pretrained_backbone=True)   
#         self.backbone = net.backbone #256 


# #         ****mobilenet****
#         self.BACKBONE_PATH = 'torch_models/mobile_ft.pt'
#         mobilenet = torch.load(self.BACKBONE_PATH)
#         modules = list(mobilenet.children())[:-1]
#         avg_pool = torch.nn.AvgPool2d(7, stride=1)
#         flatten = Flatten()
#         self.backbone = torch.nn.Sequential(*modules, avg_pool, flatten) #1280
        
# #         ****mobilnet distance matric learning****   
#         self.BACKBONE_PATH = 'mymodels/model_000400.pth'
#         self.backbone = torch.load(self.BACKBONE_PATH) #1280    

#         self.backbone = mymodels.create('Resnet50', dim=512, pretrained=True,
#                                         model_path='mymodels/ckp_ep210.pth.tar') #[227,227]->512

#         self.backbone = BN_Inception(dim=512, pretrained=True, 
#                                      model_path='mymodels/bn_inception-52deb4733.pth')

        # fc layer
        self.fc = nn.Sequential(
            avg_pool,
            flatten
        ) # 2048
        
#         self.embed = nn.Sequential(
#             self.backbone,
#             self.fc
#         ) # 2048
    
        self.device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
        self.topk = 3
        self.threshold = 0.75
        self.feature_len = 2048

        self.sort_order_descending = False
        self.cos = nn.CosineSimilarity(dim=1, eps=1e-6)
        self.roi_upsample = nn.UpsamplingBilinear2d([224,224])
        self.roi_transform = transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                    ])
        
    def embed(self, images):
        r = self(images)
        return self.fc(r)

    def forward(self, images):
        """Extract feature vectors from input images."""
        return self.backbone(images)
#         return self.backbone(images)[2]    
    

    def makeAllReference_online(self, referenceImgDirPath):
        """
        임베딩 디비 생성... flask 서버용 
        utils.makeAllReferenceCSV 에서 csv 저장만 안함
        """
        reference_dataset = torchvision.datasets.ImageFolder(
            root=referenceImgDirPath,
            transform=transforms.Compose([
#                 transforms.Resize([224,224]),
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
            ])
        )
        reference_loader = torch.utils.data.DataLoader(
            reference_dataset,
            batch_size=32,
            num_workers=0,
            shuffle=False
        )
        
        # get all data and input to model
        _temp = []
        for data, target in tqdm(reference_loader):
            outputs = self.embed(data.to(self.device)).data # .data 안하면 메모리 오버플로남...
            _temp.append(outputs)

        represented = torch.cat(_temp, dim=0)
        # raw data label 별 평균 구해두기
        self.df_ref_online = pd.DataFrame(represented.cpu().numpy())
        self.df_ref_online['label'] = [reference_dataset.classes[i] for i in reference_dataset.targets]
        self.reference_means_online = self.df_ref_online.groupby('label').mean()
        self.dir_list_online = reference_dataset.classes

        # 즉각 임베딩 디비 생성
        self.setReferenceDataset(self.dir_list_online, self.df_ref_online, self.reference_means_online)
        return self.dir_list_online, self.df_ref_online, self.reference_means_online

    def addNewData_online(self, im_arr, label):
        """
        @params im_arr: h X w X 3 (RGB)
        @params label: "praL"
        """
        data = self.roi_transform(im_arr).unsqueeze(0)
        outputs = self.embed(data.to(self.device)).data
        new_feature = list(outputs.squeeze().cpu().numpy())
        new_feature.append(label)
        new_feature[-1]
        # concat to prev df_ref_online
        self.df_ref_online.loc[self.df_ref_online.shape[0]] = new_feature
        # revise reference_means_online 
        self.reference_means_online = self.df_ref_online.groupby('label').mean()
        # 즉각 임베딩 디비 생성
        self.setReferenceDataset(self.dir_list_online, self.df_ref_online, self.reference_means_online)
        print('[Detector]: addNewData_online -', label)
        return

    def addNewLabel_online(self, dirPath, label):   
        """
        @params dirPath: "server/vu-visor/static/images_ext"
        @params label: "praL"
        """
        finalDirPath = os.path.join(dirPath, label)
        ims = [Image.open(os.path.join(finalDirPath,i)) for i in os.listdir(finalDirPath)] # n_img X h X w X c
        ims_tensor = torch.stack([self.roi_transform(im) for im in ims])
        outputs = self.embed(ims_tensor.to(self.device)).data
        new_features =outputs.cpu().numpy() # n_img X 2048

        new_df_ref_online = pd.DataFrame(new_features)
        new_df_ref_online['label'] = label
        # add new label
        if label not in self.dir_list_online: # 중복된 이름 없을 시 새로 등록
            self.dir_list_online.append(label)
            self.dir_list_online.sort() # 반드시 소팅해서 클래스 번호 재정렬 해줘야함
        # concat to prev df_ref_online
        _df_ref_online_non_overlap = self.df_ref_online[self.df_ref_online['label']!=label] # 중복된 이전 데이터 제거
        self.df_ref_online = pd.concat([_df_ref_online_non_overlap, new_df_ref_online], ignore_index=True)
        # revise reference_means_online 
        self.reference_means_online = self.df_ref_online.groupby('label').mean()
        # 즉각 임베딩 디비 생성
        self.setReferenceDataset(self.dir_list_online, self.df_ref_online, self.reference_means_online)
        print('[Detector]: addNewLabel_online -', label)
        return 
        
    # 임베딩 디비 생성
    def setReferenceDataset(self, sample_dir_list, df_ref_feature_sampled, reference_means_sampled):
        self.reference_classes = sample_dir_list
        self.reference_targets = list(df_ref_feature_sampled.iloc[:,-1])
        self.embedded_features_cpu = torch.tensor(df_ref_feature_sampled.iloc[:,:-1].as_matrix()).float().data # float64->float32(torch default), cpu
        self.embedded_features = self.embedded_features_cpu.to(self.device) # gpu
        self.embedded_means_numpy = reference_means_sampled.as_matrix()
        self.embedded_means = torch.tensor(self.embedded_means_numpy).float().data.to(self.device) # float64->float32(torch default), gpu

        self.c2i = {c:i for i,c in enumerate(self.reference_classes)}
        self.embedded_labels = torch.tensor([self.c2i[c] for c in self.reference_targets]).to(self.device)
        
        # make datafames for plot
        # ***필요없음 어차피 pca 해야함
        self.df_ = pd.DataFrame(np.array(self.embedded_features_cpu))
        self.df_['name'] = self.reference_targets
        self.centers_ = pd.DataFrame(self.embedded_means_numpy)
        self.centers_['name'] = self.reference_classes

    def fit_pca(self, n_components=4):
        # pca model fit
        self.n_components = n_components
        self.pca = PCA(n_components=self.n_components) # 2048 차원 다쓰면 나중에 샘플링에서 계산 오류남, sample 개수보다 많으면 촐레스키 분해 에러나는듯
        self.transformed = self.pca.fit_transform(self.embedded_features_cpu)

        # show PCA features 
        self.df = pd.DataFrame(self.transformed)
        self.df['name'] = self.reference_targets
        self.centers = pd.DataFrame(self.pca.transform(self.embedded_means_numpy))
        self.centers['name'] = self.reference_classes

    def inference_tensor3(self, inputs, metric='cos', knn=True):
        """
        roi 피처맵을(nxcx7x7) 인풋으로 받음
        @params metric: [l2, mahalanobis, prob]
        @params inputs: # N x C x H x W
        """            
        # inputs shape: Batch*C*H*W
        # input to backbone model
        self.inputs = inputs
        self.outputs = self.fc(self.inputs).data # n_roi X features 2048    

        # inference
        self.sort_order_descending = True         
        if knn: self.dists = my_cos(self.outputs, self.embedded_features)
        else: self.dists = my_cos(self.outputs, self.embedded_means)

        self.dists_sorted = self.dists.sort(dim=1, descending=self.sort_order_descending)
        if knn: self.predicts = self.embedded_labels[self.dists_sorted.indices][:,:self.topk]
        else: self.predicts = self.dists_sorted.indices[:,:self.topk]

        self.predicts_dist = self.dists_sorted.values[:,:self.topk]
        return self.predicts.data, self.predicts_dist.data    

    def inference_tensor_texture(self, inputs, metric='cos', knn=True, SUM=True):
        """
        현재 미니배치 1인 경우만 작동
        roi 피처맵을(nxcx7x7) 인풋으로 받음
        @params metric: [l2, mahalanobis, prob]
        @params inputs: # N x C x H x W
        """            
        # inputs shape: Batch*C*H*W
        # input to backbone model
        # 1 X 2048 X 7 X 7 --> 7*7 X 2048 --> 1 X 3
        # n X 2048 X 7 X 7 --> n*7*7 X 2048 --> n X 3
        self.outputs = inputs.data.transpose(1,2).transpose(2,3).reshape(-1,inputs.shape[1])
        # self.outputs = inputs.squeeze().transpose(0,1).transpose(1,2).reshape(-1,inputs.shape[1]).data 
        # 7*7 X features 2048   
        
        # inference
        self.sort_order_descending = True         
        if knn: self.dists = my_cos(self.outputs, self.embedded_features)
        else: self.dists = my_cos(self.outputs, self.embedded_means)

        self.dists_sorted = self.dists.sort(dim=1, descending=self.sort_order_descending)
        if knn: self.predicts = self.embedded_labels[self.dists_sorted.indices][:,:self.topk]
        else: self.predicts = self.dists_sorted.indices[:,:self.topk]

        self.predicts_dist = self.dists_sorted.values[:,:self.topk]

        if(SUM):
    #             ---- dist sum mode ----
            aa = pd.DataFrame(list(zip(self.predicts[:,0].cpu().numpy(), self.predicts_dist[:,0].cpu().numpy())), columns=['label','dist'])
            self.aa = aa
            # pp = aa.groupby('label').sum()
            # pp = pp.sort_values(by='dist', ascending=False)
            # pp = list(pp.index[:3])
            # if(len(pp)!=self.topk):
            #     pp = pp*self.topk
            #     pp = pp[:self.topk]

            # 배치별 번호 매기기
            self.aa['batch'] =  np.repeat(np.arange(inputs.shape[0]), inputs.shape[2]*inputs.shape[3])
            # 다중 그루핑
            zz = self.aa.groupby(['batch', 'label']).agg(['sum', 'mean', 'max'])
            # 2중 컬럼 인덱스 해제
            zz.columns = zz.columns.droplevel(0) 
            # sum 순으로 정렬
            zz = zz.reset_index().sort_values(by=['batch', 'sum'], ascending=False).set_index(['batch', 'label'])
            # 배치별 가장 큰 값만 리턴, dist는 선택 레이블의 평균값
            preds = zz.groupby('batch').head(1).sort_index(0).index.get_level_values(level=1).tolist()
            preds = np.repeat(preds, 3).reshape(-1,3)
            return torch.tensor(preds).data
        else:
    #             ---- voting mode ----
            c = Counter(torch.cat([*self.predicts]).cpu().numpy())
            self.c = c
            pp = list(zip(*(c.most_common()[:3])))[0]
            if(len(pp)!=self.topk):
                pp = pp*self.topk
                pp = pp[:self.topk]    
            return torch.tensor([pp]).data      

    
    def inference_tensor4(self, inputs):
        """
        roi 피처맵을(nxcx7x7) 인풋으로 받음
        """            
        # inputs shape: Batch*C*H*W
        # input to backbone model
        self.inputs = inputs
        self.outputs = self.fc(self.inputs).data # n_roi X features 2048    

        # inference
        self.dists = my_cos(self.outputs, self.embedded_means)
        self.predicts_dist = self.dists
        return self.predicts_dist.data    

    # 각 레이블 별 평균과의 L2 거리 계산 (PCA 적용안함)
    def calc_l2(self, label):
        mu = self.embedded_means[label]
        xx = self.outputs

        # 20*1280 X 1280*20 --diag--> 20
        # num_roi*dim X dim*num_roi --diag--> num_roi
        xx_sub_mu = xx.sub(mu)
        l2_dist = xx_sub_mu.matmul(xx_sub_mu.t()).sqrt().diag()
        return l2_dist.cpu()

    # 각 레이블 별 평균과의 cosine simility 계산 (PCA 적용안함)
    def calc_cos(self, label):
        mu = self.embedded_means[label]
        xx = self.outputs

        # 20*1280 X 1280*20 --diag--> 20
        # num_roi*dim X dim*num_roi --diag--> num_roi
        mu = mu.unsqueeze(0)
        cos_dist = self.cos(xx, mu)
        return cos_dist.cpu()        

    def inference_file(self, imgPath):              
        # imgPath = "./server/oracle_proj/predict.jpg"
        frame = cv2.imread(os.path.join(imgPath), cv2.IMREAD_COLOR)
        return self.inference_tensor(frame)

    def show_img(self, imgPath):
        frame = cv2.imread(os.path.join(imgPath), cv2.IMREAD_COLOR)
        # opencv frme input // H*W*C(BGR)
        # 0-255 3 channel
        inputs = torch.Tensor(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))/255 # 여기 Tensor 소문자로 바꾸면 안됨... 차이 알아보기
        plt.imshow(inputs)
        plt.show()
        
    def show_tensor(self, t):
        plt.imshow(t.numpy().transpose([1,2,0]))
        plt.show()
        
    def plot(self):
        self.fit_pca(n_components=4)
        _outputs = self.pca.transform(self.outputs.cpu())

        plt.figure(figsize=(18,9))
        plt.title("Vector space, pca_n: "+str(self.feature_len))
        ax = sns.scatterplot(x=0, y=1, hue='name', data=self.df, palette="Set1", legend="full")
        ax2 = sns.scatterplot(x=0, y=1, hue='name', data=self.centers, palette="Set1", s=150, legend=None, edgecolor='black')
        plt.scatter(_outputs[:,0], _outputs[:,1], marker='x', c='black')
        plt.show()

    # save plot image for web
    def save_plot(self, plotPath):
        self.fit_pca(n_components=4)
        _outputs = self.pca.transform(self.outputs.cpu())

        plt.figure(figsize=(9,9))
        plt.title("Vector space, pca_n: "+str(self.feature_len))
        ax = sns.scatterplot(x=0, y=1, hue='name', data=self.df, palette="Set1", legend="full", s=30)
        ax2 = sns.scatterplot(x=0, y=1, hue='name', data=self.centers, palette="Set1", s=150, legend=None, edgecolor='black')
        plt.scatter(_outputs[:,0], _outputs[:,1], marker='x', c='black', s=120)
        plt.savefig(plotPath)

In [15]:
model = OracleModel()
model.to(model.device)
model.eval()

OracleModel(
  (backbone): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
          (0): Conv2d

In [16]:
"""
차후 util로 리팩토링
"""
import PIL
from collections import Counter 
# -------------- 전체 임베딩 피처 디비 생성 --------------
def makeAllReferenceCSV(referenceImgDirPath, featuresPath, meanPath, n_sample):
    """
    전체 임베딩 피처 dataframe 생성 후 csv로 저장
    """
    reference_dataset = torchvision.datasets.ImageFolder(
        root=referenceImgDirPath,
        transform=transforms.Compose([
#             transforms.RandomRotation(90, expand=True),
            transforms.Resize([224,224]),
#             transforms.RandomHorizontalFlip(),
#             transforms.RandomRotation(45),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
    )
    reference_loader = torch.utils.data.DataLoader(
        reference_dataset,
        batch_size=32,
        num_workers=0,
        shuffle=False
    )

    # get all data and input to model
    _temp = []
    for data, target in tqdm(reference_loader):
        outputs = model.embed(data.to(model.device)).data # .data 안하면 메모리 오버플로남...
        _temp.append(outputs)       

    represented = torch.cat(_temp, dim=0)
    # raw data label 별 평균 구해두기
    df_ref = pd.DataFrame(represented.cpu().numpy())
    df_ref['label'] = [reference_dataset.classes[i] for i in reference_dataset.targets*1]
    
    # sampling
    df_ref_sampled = df_ref.groupby('label').apply(pd.DataFrame.sample, n=n_sample).reset_index(drop=True)
    reference_means = df_ref_sampled.groupby('label').mean()

    # 자료 저장
    df_ref_sampled.to_csv(featuresPath, encoding='utf8', index=False)
    reference_means.to_csv(meanPath, encoding='utf8', index=True)
    return


def set_sample_val_img(imgValPath, imgValTempPath, n_class_for_val, musthave_list=[]):
    """
    n개 클래스만 validation 샘플링 하는 함수
    @param imgValPath = '.\\img_real\\val'
    @param imgValTempPath = '.\\img_real\\_val_temp'
    @param n_class_for_val = 20
    @musthave_list = 반드시 포함할 리스트
    """
    all_dir_list = os.listdir(imgValPath)
    sample_dir_list = np.random.permutation(all_dir_list)[:n_class_for_val]
    for i in musthave_list:
        if i not in sample_dir_list: sample_dir_list = np.concatenate([sample_dir_list, [i]])

    # 추출될 이미지 저장 폴더 비우기... 비동기 버그 때문에 sleep넣음 
    if os.path.isdir(imgValTempPath): shutil.rmtree(imgValTempPath)
    sleep(0.1)
    os.makedirs(imgValTempPath)

    # copy sampled dir
    for class_name in tqdm(sample_dir_list):
        if not os.path.isdir(os.path.join(imgValTempPath, class_name)):
            os.mkdir(os.path.join(imgValTempPath, class_name))
        command = ['cmd', '\/c', 'copy',  os.path.join(imgValPath, class_name) , os.path.join(imgValTempPath, class_name)]    
        subprocess.check_output(command)
    sample_dir_list.sort() # 토치 데이터로더도 자동으로 이름순 소팅하므로 꼭 해야함
    return list(sample_dir_list)

def get_sample_reference(sample_dir_list, featuresPath, meanPath, showData=False):
    """
    n개 클래스 임베딩 & 레이블 & means 추출 
    """
    # load saved embedding features & means
    df_ref = pd.read_csv(featuresPath, encoding='utf8')
    reference_means = pd.read_csv(meanPath, encoding='utf8', index_col='label')
    
    if showData:
        display(df_ref)
        display(reference_means)

    df_ref_featere_sampled = None
    for c in sample_dir_list:
        _df = df_ref[df_ref['label'] == c]
        if df_ref_featere_sampled is None: df_ref_featere_sampled = _df
        else: df_ref_featere_sampled = df_ref_featere_sampled.append(_df)

    df_ref_featere_sampled = df_ref_featere_sampled.reset_index(drop=True)
    reference_means_sampled = reference_means.loc[sample_dir_list]
    return df_ref_featere_sampled, reference_means_sampled


def calcWindowSlidePreds(target_all, preds_all, n_window=7, mode="first"):
    """
    target_all = 정답 리스트
    preds_all = 예측값 리스트 
    mode = 'all' # first, all
    n_window = 3 
    @return new_preds_all = []
    """
    
    # preds, gt 모아서 데이터프레임화
    df = pd.DataFrame(preds_all.numpy())
    df['gt']= target_all.numpy()

    # 클래스별 인덱스 시작점 계산. 데이터로더 셔플하면 안됨
    df_agg = df.groupby('gt').count()
    df_agg['end_idx'] = df_agg[0].cumsum()

    # 윈도 내 데이터 voting
    startidx = 0
    new_preds_all = []
    for i in df_agg.index:
        for idx in range(startidx, df_agg['end_idx'][i]):
            bookmarkidx = idx-n_window+1 if idx-n_window>=0 else 0
            if idx-n_window<startidx:
                bookmarkidx = startidx

            if mode=='all': # 1~3위 까지 전체 사용 모드, [::-1] 은 가장 최근 값을 맨 앞으로 보내려고
                res = np.concatenate(preds_all[bookmarkidx:idx+1].numpy()[::-1])
            elif mode=='first': # 1위만 사용모드       
                res = preds_all[bookmarkidx:idx+1, 0].numpy()[::-1]
            res_cnt = Counter(res) # counter는 먼저나온 값이 먼저 등록됨, 즉 카운터가 동률이면 최신값이 가장 먼저 나옴
#             print(idx-n_window, idx, bookmarkidx, idx+1, res, res_cnt.most_common()[:2], '[Pred]:', res_cnt.most_common()[0][0])
            new_preds_all.append(res_cnt.most_common()[0][0])
#         print('-------------------------')
        startidx = df_agg['end_idx'][i]
    return new_preds_all

In [6]:
# ------------- non max suppression -------------
# https://wns349.github.io/2018/10/16/nms/
from itertools import combinations, permutations
# Calc IOU
def calcIOU(box1, box2):
    """
    box - x,y,w,h
    """
    area_box1 = box1[2]*box1[3]
    area_box2 = box2[2]*box2[3]
    x1_max = max(box1[0], box2[0])
    x2_min = min(box1[0]+box1[2], box2[0]+box2[2])
    y1_max = max(box1[1], box2[1])
    y2_min = min(box1[1]+box1[3], box2[1]+box2[3])
    
    area_intersection = max(0, x2_min-x1_max) * max(0, y2_min-y1_max)
    area_union = area_box1+area_box2-area_intersection +1e-9
    return area_intersection/area_union

def non_max_sup_one_class(bboxes, threshold=0.2, descending=False):
    """
    @params threshold - 
    @params ascending - 기본이 내림차순,
    """
    bboxes = list(bboxes)
    bboxes.sort(key = lambda x: x[2], reverse=descending) # 거리값이므로 오름차순, 확률이면 내림차순  
    bboxes = np.array(bboxes)
    keeps = [True]*len(bboxes)

    for i, bbox in enumerate(bboxes):
        if not keeps[i]: continue
        for j in range(i+1, len(bboxes)):
            if not keeps[i]: continue
            iou_res = calcIOU(bbox[0], bboxes[j][0])
            if iou_res>threshold: keeps[j] = False
    return bboxes[keeps]

def rpn3(box, n_slice_x, n_slice_y):
    """
    n분할 rpn 조합
    @params box: x,y,w,h
    @params im: nomalized image tensor N x C x W x H
    @return 
    """
    w = box[2]/n_slice_x
    h = box[3]/n_slice_y
    
    rois = []
    boxes = []
    
    coords = np.array(np.meshgrid(list(range(n_slice_x+1)), list(range(n_slice_y+1)))).T.reshape(-1,2)
    
    for a,b in combinations(coords,2):
        if a[0]>=b[0] or a[1]>= b[1]: continue # 넓이 없는 사각형 제거
#         print(a,b)
        boxes.append([box[0]+a[0]*w, box[1]+a[1]*h, 
                      (b[0]-a[0])*w, (b[1]-a[1])*h]) # x,y,w,h

    boxes = torch.tensor(boxes)
    return boxes

def get_rois(images, featuremaps, bboxes):
    """
    roi-align from feature map
    @params images - 원본 이미지 (사이즈 계산용)
    @params featuremaps - CNN Backbone 거쳐 나온 것
    @params bboxes - 원본 이미지에서의 bboxes (x,y,w,h)
    @return 피처맵에서 스케일된 bbox부분 7x7로 roi-align된 피처맵
    """
    # calc bbox ratio
    ratio_y = featuremaps.shape[2]/images.shape[2]
    ratio_x = featuremaps.shape[3]/images.shape[3]
    bboxes_scaled = bboxes.clone()#.detach()

    bboxes_scaled[:,0] = bboxes_scaled[:,0]*ratio_x # for x
    bboxes_scaled[:,1] = bboxes_scaled[:,1]*ratio_y # for y
    bboxes_scaled[:,2] = bboxes_scaled[:,2]*ratio_x # for w
    bboxes_scaled[:,3] = bboxes_scaled[:,3]*ratio_y # for h

    # x,y,w,h -> x1, y1, x2, y2 그래디언트 학습되는 변수가 아니므로 inplace 계산 들어가도 괜찮다
    bboxes_scaled[:, 2] = bboxes_scaled[:, 0] + bboxes_scaled[:, 2]
    bboxes_scaled[:, 3] = bboxes_scaled[:, 1] + bboxes_scaled[:, 3]
    
    crops = torchvision.ops.roi_align(featuremaps, [bboxes_scaled], [7,7])
    return crops

# -------------------- edgebox RPN ----------------------------
# https://github.com/opencv/opencv_contrib/blob/master/modules/ximgproc/samples/edgeboxes_demo.py
# https://donghwa-kim.github.io/EdgeBoxes.html - 엣지박스 간단 설명

edge_model = ".\\server\\vu-visor\\model.yml.gz"
edge_detection = cv2.ximgproc.createStructuredEdgeDetection(edge_model)

def rpn(im_opencv, num_boxs, scale=1, min_score=0.01):
    """
    region proposal network
    """
    global edge_detection
    
    def makeEdgeBox(scale):
        im_opencv_scaled = cv2.resize(im_opencv, (int(im_opencv.shape[1]*scale), int(im_opencv.shape[0]*scale)), 
                                      interpolation=cv2.INTER_CUBIC).astype(np.float32)

        edges = edge_detection.detectEdges(im_opencv_scaled / 255.0)
        orimap = edge_detection.computeOrientation(edges)
        edges = edge_detection.edgesNms(edges, orimap)
        
        edge_boxes = cv2.ximgproc.createEdgeBoxes() 
        edge_boxes.setBeta(0.75) # beta=0.1, nms threshold for object proposals.
        edge_boxes.setMaxBoxes(num_boxs)
        edge_boxes.setMinScore(min_score) # box score
        boxes, scores = edge_boxes.getBoundingBoxes(edges, orimap)
        return boxes, scores, im_opencv_scaled
    
    (boxes, scores, im_opencv_scaled) = makeEdgeBox(scale)
    # bbox 하나도 없으면 전체샷이라도 저장
    if len(boxes)==0: 
        boxes=np.array([[0,0,im_opencv_scaled.shape[1],im_opencv_scaled.shape[0]]])
        scores=np.array([[min_score]])
    boxes = (boxes/scale).round().astype(np.int)
    # 박스 개수 절반보다 모자라면 스케일 키워서 한번더
    if len(boxes)<(num_boxs/2):
        scale = scale*2
        (_boxes, _scores, im_opencv_scaled) = makeEdgeBox(scale)
        if len(_boxes)>0:
            _boxes = (_boxes/scale).round().astype(np.int)
            boxes = np.concatenate([boxes,_boxes])[:num_boxs] # 이전 스케일의 박스와 concat
            scores = np.concatenate([scores,_scores])[:num_boxs]

    return boxes, scores


# im = cv2.imread(".\\server/vu-visor/predict.jpg")
# %time boxes, scores = rpn(im, num_boxs=10, scale=1, min_score=0.01)
# for idx, (b, s) in enumerate(zip(boxes, scores)):
#     x, y, w, h = b
#     cv2.rectangle(im, (x, y), (x+w, y+h), (0, 255, 0), 1, cv2.LINE_AA)
#     cv2.putText(im, str(s), (x+3, y+17), cv2.FONT_HERSHEY_SIMPLEX, .6, (0,0,225), 2)                             

# # # cv2.imshow("edges", edges);
# cv2.imshow("edgeboxes", im)
# cv2.waitKey(0)
# cv2.destroyAllWindows()  

def calc_tp_state(boxes_ans, boxes_model, confidiences_model, filename=None):
    """
    average precision 계산용 데이터 프레임 리턴
    @params boxes - x,y,w,h
    state_array = [[filename, conf, state, iou], [filename, conf, state, iou], ...]
    """
    threshold_IOU = 0.5 # 이걸 넘어야 TP, 못넘으면 FP
    state_array = []
    for model_box_idx, model_box in enumerate(boxes_model):
        state = [filename, confidiences_model[model_box_idx], 0, 0] # default가 FP
        for ans_box_idx, ans_box in enumerate(boxes_ans):
            _IOU = float(calcIOU(model_box, ans_box))
            if _IOU>threshold_IOU: # IOU넘는게 하나라도 있으면 TP인 경우
                state = [filename, confidiences_model[model_box_idx], 1, _IOU] # default가 FP
                break
            if _IOU>state[3]: state[3]=_IOU # FP경우에도 최고 IOU기록
        state_array.append(state)
    _df = pd.DataFrame(state_array, columns=['image', 'confidience', 'TP', 'IOU'])
    return _df

from collections import Counter
# https://github.com/rafaelpadilla/Object-Detection-Metrics

def calc_AP(dataset_classes, label, target_all, dfs):
    target = dataset_classes.index(label)
    n_gt = Counter(target_all)

    _df = dfs[label].sort_values(by='confidience', ascending=False) # sort by conf

    _df['FP'] = [0 if tp==1 else 1 for tp in _df['TP']]
    _df['cum_TP'] = _df['TP'].cumsum()
    _df['cum_FP'] = _df['FP'].cumsum()
    _df['precision'] = [tp/(tp+fp) for tp,fp in zip(_df['cum_TP'],_df['cum_FP'])]
    _df['recall'] = _df['cum_TP']/n_gt[target]
    _df['precision_interpolated'] = [_df['precision'][i:].max() for i in range(len(_df['recall']))] 
    _df.reset_index(inplace=True)

    precisions = [_df['precision_interpolated'][_df['recall'].between(0.1*(i-1),0.1*i,inclusive=True)].min() for i in range(11)]
    precisions[0] = 1
    precisions = np.array([0 if p is np.nan else p for p in precisions])
    AP = precisions.mean()
#     print('[class]:', label, ', [AP]:', AP)
#     print('[Precisions]', precisions)

#     plt.plot(_df['recall'], _df['precision'])
#     plt.plot(_df['recall'], _df['precision_interpolated'], color='r')
#     plt.grid()
#     plt.xlim([0,1]), plt.title(label+' 11 point p-r curve')
#     plt.xlabel('recall'), plt.ylabel('precision')
#     plt.show()

    # display(_df)
    return AP, _df

In [23]:
def doClassification():
    # ---------------------------- validation set 로드 ---------------------------------

    val_dataset = torchvision.datasets.ImageFolder(
        root=imgValPath,
        transform=transforms.Compose([
            transforms.Resize([224,224]),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
    )

    val_loader = torch.utils.data.DataLoader(
        val_dataset,
        batch_size=32,
        num_workers=4,
        shuffle=True
    )

    # -------------- 레퍼런스 데이터에서 n개 클래스 임베딩 & 레이블 & means 추출 -----------------
    featuresPath = 'cosmetic_features_cos.csv'
    meanPath = 'cosmetic_means_cos.csv'

    sample_dir_list = val_dataset.classes
    df_ref_featere_sampled, reference_means_sampled = get_sample_reference(sample_dir_list, featuresPath, meanPath, showData=False)

    # ---------------------------- 레퍼런스셋 로드 및 임베딩 세팅 ---------------------------------
    model.setReferenceDataset(val_dataset.classes, df_ref_featere_sampled, reference_means_sampled)

    # ---------------------------- validation set 정확도 측정 ---------------------------------
    print('**********classification***********')

    target_all = []
    preds_all = []
    for data, target in tqdm(val_loader):
        data = data.to(model.device)
        preds, preds_dist = model.inference_tensor3(model(data).data, 'cos', knn=USE_KNN)
        target_all.append(target)
        preds_all.append(preds)
        

    target_all=torch.cat(target_all, dim=0)
    preds_all=torch.cat(preds_all, dim=0).data.cpu()

    return target_all, preds_all

def doBboxClassification():
    global detect_ref_dir, class_path
    print('********** [n_sample]:',n_sample,'[ref]:',ref,'[val]:',val,'[knn]:',USE_KNN,'[texture]:',USE_TEXTURE,'[threshold]:',model.threshold,'***********')

    detect_acc_dataset = MyDataset(detect_ref_dir, class_path, 
                          transform=transforms.Compose([
    #         transforms.Resize([224,224]),
    #         transforms.RandomHorizontalFlip(),
    #         transforms.RandomRotation(45),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
    )

    # ---------------------------- val set 정확도 측정 ---------------------------------
    print('**********classification-bbox***********')
    target_all = []
    preds_all = []
#     for i in tqdm(np.random.permutation(len(detect_acc_dataset))):
    for i in tqdm(range(len(detect_acc_dataset))):
        im_tensor, targets_gt, boxes_gt = detect_acc_dataset[i]
        featuremaps = model(im_tensor.to(model.device))
        target_all = np.concatenate([target_all,  targets_gt])
        
        _boxes_cuda = torch.from_numpy(boxes_gt).float().cuda()
        rois = get_rois(im_tensor, featuremaps, _boxes_cuda)
        if(USE_TEXTURE): preds = model.inference_tensor_texture(rois, 'cos', knn=USE_KNN, SUM=True)
        else: preds, preds_dist= model.inference_tensor3(rois, 'cos', knn=USE_KNN)
        preds_all.append(preds.data)

    target_all=torch.tensor(target_all).data.int()
    preds_all=torch.cat(preds_all, dim=0).data.int().cpu()
    
    # groupby 통계량 보기
    _res = []
    for t,p in zip(target_all.numpy(), preds_all.numpy()):
        _res.append([detect_acc_dataset.classes[t],t,p,t==p[0], t in p])
    _df_res = pd.DataFrame(_res, columns=['label', 'target', 'pred', 'top1', 'top3'])
    res_agg_df = _df_res.groupby('label').mean()

    return target_all, preds_all, res_agg_df['top1'].as_matrix()

def doDetection():
    global detect_ref_dir, class_path
    print('**********detection***********')
    # ---------------------------- val set 로드 ---------------------------------
    voc_val_dataset = MyDataset(detect_ref_dir, class_path, 
            transform=transforms.Compose([
            transforms.ToTensor()
        ]))

    voc_transform = transforms.Compose([
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
    # assert val_dataset.classes==sample_dir_list, "샘플링 클래스와 데이터로더 클래스가 다름"
    # ---------------------------- val set 정확도 측정 ---------------------------------

    fps=0
    # model.threshold = 0.9*0.08 #0.85 #0.4
    SHOW_IMAGE = False # 이미지 볼지 말지
    dfs = {c:pd.DataFrame(columns=['image', 'confidience', 'TP', 'IOU']) for c in voc_val_dataset.classes} # average precision 저장
    target_all = [] #ground truth bboxs

    # print('[num_class]:', len(sample_dir_list), sample_dir_list)
#     for i in tqdm(np.random.permutation(len(voc_val_dataset))):
    for i in tqdm(range(len(voc_val_dataset))):
        startTime = time.time()
        data, targets_gt, boxes_gt = voc_val_dataset[i] # 정답 데이터, 레이블, bbox

        frame = data.clone().detach().mul(255).squeeze().numpy().astype(np.uint8).transpose([1,2,0])
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB) # opencv image need to convert BGR -> RGB

        im_tensor = voc_transform(data.squeeze()).to(model.device).data.unsqueeze(0)
        featuremaps = model(im_tensor)
        target_all = np.concatenate([target_all,  targets_gt])
            
        # region proposal network extracts ROIs
        boxes, scores = rpn(frame, num_boxs=n_rpn_box, scale=1, min_score=0.01)
        cv2.putText(frame, str(len(boxes)), (data.shape[-1]-80,30), cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(0,100,225), thickness=2) # rpn생성 box 개수
        
        if SHOW_IMAGE:    
            # render ground truth roi prediction
            cv2.putText(frame, str(float(preds_dist.mean()))[:5], (20,30), cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(0,200,225), thickness=2) # roi평균 유사도
            _boxes_gt_cuda = torch.from_numpy(boxes_gt).float().cuda()
            rois_gt = get_rois(im_tensor, featuremaps, _boxes_gt_cuda)
            preds_gt, pred_dists_gt = model.inference_tensor3(rois_gt, 'cos', knn=USE_KNN)    
            for pred, dist, b in zip(preds_gt[:,0], pred_dists_gt[:,0], boxes_gt):
                pred_label = voc_val_dataset.classes[pred]
                res_text = "("+str(float(dist))[:5]+")"+pred_label
                x, y, w, h = b
                cv2.putText(frame, res_text, (x+3, y+12), cv2.FONT_HERSHEY_SIMPLEX, fontScale=.5, color=(255,0,255), thickness=1)
                cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 1, cv2.LINE_AA)  
            
        # inference scores, boxes
        _boxes_cuda = torch.from_numpy(boxes).float().cuda()
        rois = get_rois(im_tensor, featuremaps, _boxes_cuda)
        preds, preds_dist= model.inference_tensor3(rois, 'cos', knn=USE_KNN)
        
    #     # make new scores
    #     scores = torch.from_numpy(scores).to(model.device)
    #     preds_dist = preds_dist*scores
        
        # objectness filterling
        filter_idx = (preds_dist[:,0]>model.threshold).type(torch.bool).cpu()
        if any(filter_idx): # 필터 통과하는거 하나라도 있어야 함
            _boxes_cuda = _boxes_cuda[filter_idx]
            preds = preds[filter_idx]
            preds_dist = preds_dist[filter_idx]
            rois = rois[filter_idx]
            
            for i in range(1):
                boxes_cuda = _boxes_cuda.clone().detach()
                if i==1:
                    # UBBR adjusted bboxes
                    offsets = UBBR_model.fc(rois) 
            #                 offsets = UBBR_model(im_tensor, offsets)        
                    # reg 적용한 random boxes
                    boxes_cuda = regression_transform(boxes_cuda, offsets)

                # non-maximum-suppression
                bboxes_all = np.array(list(zip(boxes_cuda.cpu(), preds.cpu().numpy()[:,0], preds_dist.cpu().numpy()[:,0])), dtype=np.object)
                bboxes_all_nms = []
                for cls in set(bboxes_all[:,1]):
                    bboxes_all_nms.append(non_max_sup_one_class(bboxes_all[bboxes_all[:,1]==cls], threshold=0.05, descending=model.sort_order_descending))
                bboxes_all_nms = np.concatenate(bboxes_all_nms)

                # render frame
                if SHOW_IMAGE:
                    for idx, (box, pred, dist) in enumerate(bboxes_all_nms):
                        pred_label = model.reference_classes[pred]
                        res_text = "("+str(float(dist))[:5]+")"+pred_label
                        x, y, w, h = box
                        if i==1: cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 0, 255), 1, cv2.LINE_AA) # UBBR
                        else: cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 255), 1, cv2.LINE_AA)
                        cv2.putText(frame, res_text, (x+3, y+12), cv2.FONT_HERSHEY_SIMPLEX, .5, (0,0,225), 1)        

            # collect statistics for mAP
            # ans data - targets_gt, boxes_gt, filename_gt
            for t in set(bboxes_all_nms[:,1]): # iterate all class
                bboxes_all_nms_per_class = bboxes_all_nms[bboxes_all_nms[:,1]==t]
                boxes_gt_per_class = boxes_gt[targets_gt==t]

                boxes_model = bboxes_all_nms_per_class[:,0]
                confidiences_model = bboxes_all_nms_per_class[:,2]
                _df = calc_tp_state(boxes_gt_per_class, boxes_model, confidiences_model, filename='filename_gt') # class 하나만 가정, 
                _class = voc_val_dataset.classes[t]
                dfs[_class] = dfs[_class].append(_df, ignore_index=True) # update df 
        
        # only render UI
        if SHOW_IMAGE:
            # frame = cv2.resize(frame, (int(frame.shape[1]*1.5), int(frame.shape[0]*1.5)), interpolation=cv2.INTER_CUBIC)
            cv2.imshow('frame',frame)
            k = cv2.waitKey(0) & 0xff 
            if k == 27: # esc
                break

        endTime = time.time()
        fps = int(1/(endTime - startTime))
            
    cv2.destroyAllWindows()        

    # calc AP, mAP, TPs, FPs
    AP_list = []
    df_res_list = {}
    for cls in voc_val_dataset.classes:
        AP, df_res = calc_AP(voc_val_dataset.classes, cls, target_all, dfs)
        AP_list.append(AP)
        df_res_list[cls] = df_res
    AP_list = np.array(AP_list)

    mAP = AP_list.mean()

    tps = []
    fps = []
    for k in df_res_list:
        try:
            tps.append(list(df_res_list[k]['cum_TP'])[-1])
            fps.append(list(df_res_list[k]['cum_FP'])[-1])
        except:
            tps.append(np.nan)
            fps.append(np.nan)

    return mAP, AP_list, fps, tps

In [10]:
allpath = {
  'refPath':{
    # 이건 floor, frontview, 
    'trimed-frontview':'.\\new-cosmetic-frontview\\images_bbox',
    'trimed-floor':'.\\new-cosmetic-floor\\images_bbox',
    'trimed-office':'.\\new-cosmetic-office\\images_bbox',
    'original-frontview':'.\\new-cosmetic-frontview\\images_ext',
    'original-floor':'.\\new-cosmetic-floor\\images_ext',
    'original-office':'.\\new-cosmetic-office\\images_ext',

    'original': '.\\references\\original',
    'full': '.\\references\\full',
    'no-masked-trimed': '.\\references\\images_trimed',
    'no-masked-trimed-10shot':'C:\\Users\\LSW\\Downloads\\mask_img\\images_trimed_10shot_class',
    'masked-trimed':'C:\\Users\\LSW\\Downloads\\mask_img\\masked_images_trimed_class',
  },
  'clsValPath':{
    #   for acc-full
    'frontview2':'.\\new-cosmetic-frontview2\\images_full',
    'wall':'.\\new-cosmetic-wall\\images_full',
    'floor2':'.\\new-cosmetic-floor2\\images_full',
    # 'office':'.\\new-cosmetic-office\\images_full',
  },
  'detectionValPath':{
    #   for acc-bbox, mAP
    'frontview2':'C:\\Users\\LSW\\Desktop\\Smart_mirror\\Yolo_mark\\x64\\Release\\cosmetic-data\\new-cosmetic-frontview2\\temp',
    'wall':'C:\\Users\\LSW\\Desktop\\Smart_mirror\\Yolo_mark\\x64\\Release\\cosmetic-data\\new-cosmetic-wall\\temp',
    'floor2':'C:\\Users\\LSW\\Desktop\\Smart_mirror\\Yolo_mark\\x64\\Release\\cosmetic-data\\new-cosmetic-floor2\\temp',
    # 'office':'C:\\Users\\LSW\\Desktop\\Smart_mirror\\Yolo_mark\\x64\\Release\\cosmetic-data\\new-cosmetic-office\\temp',
  },
  'classPath':'C:\\Users\\LSW\\Desktop\\Smart_mirror\\Yolo_mark\\x64\\Release\\cosmetic-data\\obj.names'
}

cols = ['timestamp', 'ref', 'val', 'n-sample', 'knn', 'texture', 'slide', 'threshold', 'n-box', 'acc-full', 'acc-bbox', 'mAP', 'memo',
        *['AP'+str(c) for c in range(31)],
        *['FP'+str(c) for c in range(31)],
        *['TP'+str(c) for c in range(31)],
        *['ACC'+str(c) for c in range(31)]]

In [10]:
for ref in ['trimed-frontview', 'trimed-floor', 'original-frontview', 'original-office']:
    ll = os.listdir(allpath['refPath'][ref])
    for i in ll:
        ss = os.listdir(os.path.join(allpath['refPath'][ref], i))
        print(len(ss), os.path.join(allpath['refPath'][ref], i))

13 .\new-cosmetic-frontview\images_bbox\AHC-AGELESS REAL EYE CREAM FOR FACE
15 .\new-cosmetic-frontview\images_bbox\AHC-Aura Secret Toneup Cream
20 .\new-cosmetic-frontview\images_bbox\AHC-ONLY FOR MAN LOTION
13 .\new-cosmetic-frontview\images_bbox\APIEU-데카소사이드 시카 겔 데이크림
16 .\new-cosmetic-frontview\images_bbox\APIEU-스타트업 포어 프라이머
14 .\new-cosmetic-frontview\images_bbox\BANILACO-프라임 프라이머 클래식
16 .\new-cosmetic-frontview\images_bbox\BELIF-The true cream AQUA BOMB
18 .\new-cosmetic-frontview\images_bbox\CLIO-스파클링 라인 프리즘 에어 아이섀도우
15 .\new-cosmetic-frontview\images_bbox\FERRAGAMO-INCANTO CHARMS
20 .\new-cosmetic-frontview\images_bbox\FERRARI-SCUDERIA BLACK EAU DE TOILETTE SPRAY
12 .\new-cosmetic-frontview\images_bbox\FROMNATURE-에이지 인텐스 트리트먼트 에센스
13 .\new-cosmetic-frontview\images_bbox\GENABELLE-LASOR SOOTHING SUNSCREEN
14 .\new-cosmetic-frontview\images_bbox\GREEN FINGER-KIDS FACIAL LOTION
14 .\new-cosmetic-frontview\images_bbox\HERA-BLACK CUSHION
16 .\new-cosmetic-frontview\images_bbox\HOLIK

In [25]:
%%time
# 결과 로그파일 불러오기
try:
    res_df = pd.read_csv('result-log.csv')
except:
    res_df = pd.DataFrame(columns=cols)
    
# option
ref = 'masked-trimed'
val = 'wall'
# n_sample = 10
# USE_KNN = True

GET_mAP = False
USE_TEXTURE = True
n_slice = 3

SLIDE_WINDOW = True
# window_mode = 'first'

model.threshold = 0.83
n_rpn_box = 200
# memo = 'randomRot,slidwindow n=%d mode=%s'%(window_size, window_mode)

for i in range(5):
    for n_sample in [1, 3, 5, 10]:
        memo=''

#             for ref in ['trimed-floor']:
        for ref in ['trimed-frontview', 'trimed-floor', 'trimed-office', 'original-frontview', 'original-floor', 'original-office']:
            # reference data
            referenceImgDirPath = allpath['refPath'][ref]
            featuresPath = 'cosmetic_features_cos.csv'
            meanPath = 'cosmetic_means_cos.csv'
            makeAllReferenceCSV(referenceImgDirPath, featuresPath, meanPath, n_sample) # 매번 랜덤 샘플링임

            for val in ['wall', 'frontview2', 'floor2']:
                if ref==val: continue
                for USE_KNN in [True, False]:
                    for USE_TEXTURE in [True, False]:
                        print('------------------', i, n_sample, '----------------')

                        # full acc용 val data
                        imgValPath = allpath['clsValPath'][val]
                        # 디텍션용 val data
                        detect_ref_dir = allpath['detectionValPath'][val]
                        class_path = allpath['classPath']

                        # ---------------------------------------- do ALL
                        target_all, preds_all = doClassification()
                        target_all_detect, preds_all_detect, accs = doBboxClassification()

                        if(GET_mAP):
                            mAP, AP_list, fps, tps = doDetection()


                        top1_acc = target_all.eq(preds_all[:,0]).float().mean()                 
                        top1_acc_detect = target_all_detect.eq(preds_all_detect[:,0]).float().mean()
                        # saveRes()-------------------------
                        if(GET_mAP):
                            result = [str(datetime.now()), ref, val, n_sample, USE_KNN, USE_TEXTURE, False, model.threshold, n_rpn_box, top1_acc.item(),
                                      top1_acc_detect.item(), mAP, memo, *AP_list, *fps, *tps, *accs]
                        else: 
                            result = [str(datetime.now()), ref, val, n_sample, USE_KNN, USE_TEXTURE, False, model.threshold, n_rpn_box, top1_acc.item(),
                                  top1_acc_detect.item(), -1, memo, *([np.nan]*93), *accs]
                        res_df = res_df.append(pd.Series(result, index=cols), ignore_index=True)
                        res_df.to_csv('result-log.csv',index=False)
                        res_df.to_csv('result-log.bak.csv',index=False)
        #                     display(res_df.iloc[[-1],:])

                        if(SLIDE_WINDOW):   
                            # window slide 보정
                            new_preds_all = calcWindowSlidePreds(target_all, preds_all, n_window=7, mode='first')
                            new_preds_all_detect = calcWindowSlidePreds(target_all_detect, preds_all_detect, n_window=7, mode='first')
                            top1_acc = target_all.eq(torch.tensor(new_preds_all)).float().mean()
                            top1_acc_detect = target_all_detect.eq(torch.tensor(new_preds_all_detect)).float().mean()    
                            # saveRes()------------------------
                            if(GET_mAP):
                                result = [str(datetime.now()), ref, val, n_sample, USE_KNN, USE_TEXTURE, SLIDE_WINDOW, model.threshold, n_rpn_box, top1_acc.item(),
                                          top1_acc_detect.item(), mAP, memo, *AP_list, *fps, *tps, *accs]
                            else: 
                                result = [str(datetime.now()), ref, val, n_sample, USE_KNN, USE_TEXTURE, SLIDE_WINDOW, model.threshold, n_rpn_box, top1_acc.item(),
                                      top1_acc_detect.item(), -1, memo, *([np.nan]*93), *accs]
                            res_df = res_df.append(pd.Series(result, index=cols), ignore_index=True)
                            res_df.to_csv('result-log.csv',index=False)
                            res_df.to_csv('result-log.bak.csv',index=False)
    #                                 display(res_df.iloc[[-1],:])

A Jupyter Widget

Exception in thread Thread-3314:
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\threading.py", line 916, in _bootstrap_inner
    self.run()
  File "C:\ProgramData\Anaconda3\lib\site-packages\tqdm\_monitor.py", line 63, in run
    for instance in self.tqdm_cls._instances:
  File "C:\ProgramData\Anaconda3\lib\_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration




------------------ 0 1 ----------------
**********classification***********


A Jupyter Widget


********** [n_sample]: 1 [ref]: trimed-frontview [val]: wall [knn]: True [texture]: True [threshold]: 0.83 ***********
**********classification-bbox***********


...


********** [n_sample]: 1 [ref]: trimed-frontview [val]: wall [knn]: True [texture]: False [threshold]: 0.83 ***********
**********classification-bbox***********


A Jupyter Widget


------------------ 0 1 ----------------
**********classification***********


A Jupyter Widget

RuntimeError: CUDA out of memory. Tried to allocate 26.00 MiB (GPU 0; 11.00 GiB total capacity; 4.04 GiB already allocated; 20.27 MiB free; 321.06 MiB cached)

In [58]:
# %%time
# # 결과 로그파일 불러오기
# try:
#     res_df = pd.read_csv('result-log.csv')
# except:
#     res_df = pd.DataFrame(columns=cols)
    
# # option
# ref = 'masked-trimed'
# val = 'wall'
# n_sample = 10
# USE_KNN = True

# GET_mAP = True
# USE_TEXTURE = False
# n_slice = 3

# SLIDE_WINDOW = True
# window_size = 7
# window_mode = 'first'

# model.threshold = 0.83
# n_rpn_box = 200
# # memo = 'randomRot,slidwindow n=%d mode=%s'%(window_size, window_mode)

# # for i in range(100):
# #     print('---------', i, '----------')
# #     for n_sample in [30]:
# #         memo = 'finding sota'

# # for USE_TEXTURE in [True, False]:
# #     if(USE_TEXTURE): memo = "Texture "+memo
# for n_sample in [5, 10, 20, 30]:
#     memo = '' # 실행전에 트랜스포머 바꿀것

#     for ref in ['no-masked-trimed', 'original', 'full']:
#         # reference data
#         referenceImgDirPath = allpath['refPath'][ref]
#         featuresPath = 'cosmetic_features_cos.csv'
#         meanPath = 'cosmetic_means_cos.csv'
#         makeAllReferenceCSV(referenceImgDirPath, featuresPath, meanPath, n_sample) # 매번 랜덤 샘플링임

#         for val in ['wall', 'frontview2', 'floor2']:
#             if ref==val: continue
#             for USE_KNN in [True, False]:

#                 # full acc용 val data
#                 imgValPath = allpath['clsValPath'][val]
#                 # 디텍션용 val data
#                 detect_ref_dir = allpath['detectionValPath'][val]
#                 class_path = allpath['classPath']

#                 # ---------------------------------------- do ALL
#                 target_all, preds_all = doClassification()
#                 target_all_detect, preds_all_detect = doBboxClassification()

#                 if(GET_mAP):
#                     mAP, AP_list, fps, tps = doDetection()


#                 top1_acc = target_all.eq(preds_all[:,0]).float().mean()                 
#                 top1_acc_detect = target_all_detect.eq(preds_all_detect[:,0]).float().mean()
#                 # saveRes()-------------------------
#                 if(GET_mAP):
#                     result = [str(datetime.now()), ref, val, n_sample, USE_KNN, model.threshold, n_rpn_box, top1_acc.item(),
#                               top1_acc_detect.item(), mAP, memo, *AP_list, *fps, *tps]
#                 else: 
#                     result = [str(datetime.now()), ref, val, n_sample, USE_KNN, model.threshold, n_rpn_box, top1_acc.item(),
#                           top1_acc_detect.item(), -1, memo, *([np.nan]*93)]
#                 res_df = res_df.append(pd.Series(result, index=cols), ignore_index=True)
#                 res_df.to_csv('result-log.csv',index=False)
#                 res_df.to_csv('result-log.bak.csv',index=False)
# #                     display(res_df.iloc[[-1],:])

#                 if(SLIDE_WINDOW):   
#                     for window_mode in ['first', 'all']:
#                         for window_size in [3,4,5,6,7]:
#                             # window slide 보정
#                             _memo = memo + ' slidewindow n=%d mode=%s'%(window_size, window_mode)  
#                             _memo = _memo.strip()
#                             new_preds_all = calcWindowSlidePreds(target_all, preds_all, n_window=window_size, mode=window_mode)
#                             new_preds_all_detect = calcWindowSlidePreds(target_all_detect, preds_all_detect, n_window=window_size, mode=window_mode)
#                             top1_acc = target_all.eq(torch.tensor(new_preds_all)).float().mean()
#                             top1_acc_detect = target_all_detect.eq(torch.tensor(new_preds_all_detect)).float().mean()    
#                             # saveRes()------------------------
#                             if(GET_mAP):
#                                 result = [str(datetime.now()), ref, val, n_sample, USE_KNN, model.threshold, n_rpn_box, top1_acc.item(),
#                                           top1_acc_detect.item(), mAP, _memo, *AP_list, *fps, *tps]
#                             else: 
#                                 result = [str(datetime.now()), ref, val, n_sample, USE_KNN, model.threshold, n_rpn_box, top1_acc.item(),
#                                       top1_acc_detect.item(), -1, _memo, *([np.nan]*93)]
#                             res_df = res_df.append(pd.Series(result, index=cols), ignore_index=True)
#                             res_df.to_csv('result-log.csv',index=False)
#                             res_df.to_csv('result-log.bak.csv',index=False)
# #                                 display(res_df.iloc[[-1],:])

SyntaxError: invalid syntax (<unknown>, line 28)